In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers textract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=18218e55ee92f562d2c5c0053241ba9162

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
# guidlines.pdf must be uploaded.
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/guidlines.pdf")
pages = loader.load_and_split()
print(pages[0])
chunks=pages

page_content='Joel Yager, M.D.\nMichael J. Devlin, M.D.Katherine A. Halmi, M.D.David B. Herzog, M.D.James E. Mitchell III, M.D.Pauline Powers, M.D.Kathryn J. Zerbe, M.D. Guideline Watch\n(August 2012):\nPractice Guideline for\nthe Treatment of Patients With\nEating Disorders, 3rd Edition\nAmerican Psychiatric Association\nThis guideline watch reviews new evidence and high-\nlights salient developments since the 2006 publica-tion of APA ’sPractice Guideline for the Treatment\nof Patients With Eating Disorders, 3rd Edition. The\nauthors of this watch constituted the work groupthat developed the 2006 guideline. We ﬁnd the guide-\nline to remain substantially correct and current in itsr e c o m m e n d a t i o n s .T h es o l ee x c e p t i o ni sar e c o m m e n -dation (with moderate-level con ﬁdence) for sibutramine\nfor binge-eating disorder. In 2010, the U.S. Food andDrug Administration (FDA) withdrew approval forsibutramine because clinical trials showed increasedrisk of heart attack

In [ ]:
import textract
texts = textract.process("/content/guidlines.pdf")

In [ ]:
with open('guidlines.txt', 'w') as f:
    f.write(texts.decode('utf-8'))

with open('guidlines.txt', 'r') as f:
    text = f.read()


In [ ]:
#llama access key
from huggingface_hub import notebook_login
import torch
notebook_login()

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00


In [ ]:
#questions.json must be uploaded.
from datasets import load_dataset
questions = load_dataset("json", data_files="/content/questions.json")['train']



question = questions["question"]
answer = questions["answer"]
print(question)
print(answer)

[' What are the symptoms of anorexia nervosa?', ' How anorexia nervosa can be diagnosed?', ' What are the physical and social consequences of anorexia nervosa?', ' What are the symptoms of bulimia nervosa?', 'How bulimia nervosa can be diagnosed?', ' What is the physical and social consequences of bulimia nervosa?', ' What are the symptoms of binge eating disorder?', ' What are the treatment options for binge eating disorder?', 'What are the causes of binge eating disorder?', 'What are the symptoms of atypical eating disorders?', ' How atypical eating disorders can be diagnosed?', ' What are the key priorities for implementation of anorexia nervosa?', ' What are the key priorities for implementation of bulimia nervosa?', ' What are the key priorities for implementation of atypical eating disorders?', ' I am worried that I have an unhealthy relationship with food, How can I understand if I have an eating disorder.']
["Anorexia nervosa is a syndrome characterized by maintaining a low wei

In [ ]:
import pandas as pd
data={'question':[],'answer':[],'Eating Disorder Chatbot Answer':[]}
df=pd.DataFrame(data)

In [ ]:
display(df)

,question,answer,Eating Disorder Chatbot Answer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def tokenCount(text: str) -> int:
    return len(tokenizer.encode(text))


text_splitter = RecursiveCharacterTextSplitter(

    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = tokenCount,
)

chunks = text_splitter.create_documents([text])

In [ ]:
chunks[4]

Document(page_content='We also searched the Cochrane database, using\nthe terms “anorexia nervosa,” “bulimia,” and “binge eat-\ning” as well as corresponding Medical Subject Head-\nings (MeSH) for reviews published from 2003\nthrough December 13, 2011.\n\nThese search strategies yielded 1,346 articles. Of\nthese articles, 693 were rejected as not relating to\ntreatment of eating disorders. We retained and re-\nviewed 91 articles pertaining to anorexia nervosa, 84\nto bulimia nervosa, 95 to binge eating, 12 to oste-\noporosis treatment in eating disorders, and 60 to\nmiscellaneous topics, most of which covered more\nthan one eating disorder.\n\nThe following discussion focuses on randomized,\ncontrolled trials identiﬁed by our search but also in-\ncludes some recent open trials of which we are aware.\nFor some topics, instead of discussing all studies,\nin this watch we summarize the conclusions of an\navailablesystematicreview.Thiswatchisnotintended\n\ntobe a comprehensivereview ofall 

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
!pip install faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.2 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
query = "What is the signs of Aneroxia Nervosa"
docs = db.similarity_search(query)
docs[0]

Document(page_content='Mood disturbance is extremely common in bulimia nervosa and symptoms of anxiety\nand tension are frequently experienced. Self-denigratory thoughts may develop out of\ndisgust at overeating or purging whilst low self-esteem and physical self-loathing may in\nsome be rooted in the past experience of physical or sexual abuse. Self-harm, commonly\nby scratching or cutting, is common. A significant proportion of those with bulimia\nnervosa have a history of disturbed interpersonal relationships with poor impulse\ncontrol. Some will abuse alcohol and drugs. \n\n2.2.2 Diagnosis\n\nAs in anorexia nervosa, the diagnosis depends on obtaining a history supported, as\nappropriate, by the corroborative account of a parent or relative. This will require an\nempathic, supportive, non-judgemental interview style in which the person is enabled to\nreveal the extent of his or her symptoms and behaviours. Although those with bulimia\nnervosa generally have fewer serious physical co

In [ ]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=True,load_in_8bit=True )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})


In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
Please answer the query as paragraph. If you don't know the answer, just say that you don't know, don't try to make up an answer."""


In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nPlease answer the query as paragraph. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
table=pd.DataFrame(data)

In [ ]:
n=len(question)
for i in range(0, n):
  query=question[i]

  result = qa_chain(query)
  ed_chatbot_answer=result['result']
  data2={'question':[query],'answer':[referance_answer],'Eating Disorder Chatbot Answer':[ed_chatbot_answer]}
  print(ed_chatbot_answer)
  df2=pd.DataFrame(data2)
  table=pd.concat([table, df2])
display(table)

  Thank you for providing the context. To answer your question, the symptoms of anorexia nervosa are:

Self-imposed or maintained weight loss, resulting in a significantly low body weight for age and height (BMI < 17.5)

Overvaluation of shape and weight (i.e., excessive preoccupation with weight and body shape)

Distorted body image (i.e., beliefs about one's body shape or size that are not based in reality)

Restrictive eating habits (i.e., limiting food intake to a narrow range of foods or avoiding entire food groups)

Bingeing (i.e., consuming large amounts of food in a short period of time)

Psychological symptoms, such as anxiety, depression, and obsessive-compulsive behaviors

It is important to note that these symptoms can vary in severity and may not be present in all individuals with anorexia nervosa. Additionally, some individuals may exhibit different symptoms or experiences, and the diagnosis should be made by a qualified mental health professional based on a comprehensive

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Atypical eating disorders can be diagnosed through a combination of clinical evaluation and diagnostic criteria. While the diagnostic criteria for anorexia nervosa and bulimia nervosa are specific and well-defined, atypical eating disorders are characterized by a range of symptoms that may not meet the full criteria for these conditions.

The diagnosis of atypical eating disorders typically involves a comprehensive clinical evaluation, including a thorough medical and psychiatric history, a physical examination, and a psychological assessment. The clinician will look for symptoms such as over concern with weight and shape, binge eating and purging, and a distorted body image. However, the frequency and severity of these symptoms may not meet the full criteria for anorexia nervosa or bulimia nervosa.

In addition to clinical evaluation, the diagnosis of atypical eating disorders may also involve the use of standardized assessment tools, such as questionnaires or rating scales. These t

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Based on the provided context, the key priorities for implementation of anorexia nervosa are:

1. Engagement: Effective engagement of the patient in the treatment plan is a precondition for any successful psychological treatment. Healthcare professionals involved in the treatment of anorexia nervosa should take time to build an empathic, supportive, and collaborative relationship with patients and, if applicable, their carers.
2. Early intervention: The case for early intervention in anorexia nervosa is highlighted, and theoretical exploration of maintaining factors is discussed. Early intervention is crucial to prevent the development of severe psychopathology and to improve outcomes.
3. Family interventions: Family interventions are usually offered in the treatment of anorexia nervosa in children and adolescents. These interventions may vary in approach and may not resemble evidence-based family interventions that have a focus on eating behaviors.
4. Individual psychological therap

...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Based on the provided context, the key priorities for implementation of atypical eating disorders are:

1. Comprehensive assessment: Assessment of people with atypical eating disorders should be comprehensive and include physical, psychological, and social needs, as well as a comprehensive assessment of risk to self.
2. Monitoring risk: The level of risk to the patient's mental and physical health should be monitored as treatment progresses, as it may increase following weight change or at times of transition between services in cases of anorexia nervosa.
3. Awareness of indicators of abuse: Health care professionals assessing children and adolescents with atypical eating disorders should be alert to indicators of abuse (emotional, physical, and sexual) and should remain so throughout treatment.
4. Respect for confidentiality: The right to confidentiality of children and adolescents with atypical eating disorders should be respected, and health care professionals working with this po

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Dear friend,

I understand your concern about your relationship with food and the possibility of having an eating disorder. It takes a lot of courage to acknowledge and seek help for such a sensitive and complex issue.

Firstly, it's important to understand that eating disorders are not just about food, but they are a complex interplay of emotional, psychological, and cultural factors. If you identify with any of the following signs, it may indicate an unhealthy relationship with food and the possibility of an eating disorder:

1. Obsessive thoughts and behaviors around food: Do you find yourself constantly thinking about food, weighing your options, and feeling guilty or ashamed about what you eat?
2. Restrictive eating: Are you restricting your food intake, avoiding certain food groups, or feeling like you cannot eat certain foods?
3. Bingeing: Do you find yourself bingeing on food, feeling like you cannot stop once you start, or hiding food to eat in secret?
4. Purging: Are you en

,question,answer,Eating Disorder Chatbot Answer
0,What are the symptoms of anorexia nervosa?,Anorexia nervosa is a syndrome characterized b...,Thank you for providing the context. To answ...
0,How anorexia nervosa can be diagnosed?,Diagnosing anorexia nervosa in older adolesce...,Anorexia nervosa can be diagnosed through a ...
0,What are the physical and social consequences...,"Over time, emotional difficulties increase, ac...",The physical and social consequences of anor...
0,What are the symptoms of bulimia nervosa?,Bulimia nervosa is characterized by recurrent ...,The symptoms of bulimia nervosa include:\n\n...
0,How bulimia nervosa can be diagnosed?,"In diagnosing bulimia nervosa, obtaining a com...",Bulimia nervosa can be diagnosed through a c...
0,What is the physical and social consequences ...,In the long-term consequences of bulimia nerv...,The physical and social consequences of buli...
0,What are the symptoms of binge eating disorder?,The main symptom of binge eating disorder is ...,The symptoms of binge eating disorder are:\n...
0,What are the treatment options for binge eati...,Most people recover from binge eating disorder...,"Based on the provided context, there are sev..."
0,What are the causes of binge eating disorder?,The exact causes of binge eating disorder are...,Thank you for providing the context. Based o...
0,What are the symptoms of atypical eating disor...,A number of people suffer from eating disorde...,"Based on the provided context, the symptoms ..."


In [ ]:
table.to_csv('ed_chatbot.csv')

In [ ]:
generated_answers_list=table["Eating Disorder Chatbot Answer"].values.tolist()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install bert-score




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
from bert_score import score
from bert_score import BERTScorer

In [ ]:
p_total=0
recall_total=0
f1_total=0

In [ ]:
a=len(answers_list)
for b in range(0,a):
  referance_answer=answer[b]
  generated_text=generated_answers_list[b]
  scorer = BERTScorer(model_type='distilbert-base-uncased')
  P, R, F1 = scorer.score([referance_answer], [generated_text])
  p_score=P.mean()
  recall_score=R.mean()
  f1_score=F1.mean()
  print(f"BERTScore Precision: {p_score}, Recall: {recall_score}, F1: {f1_score}")
  p_total=p_total + p_score
  recall_total=recall_total+recall_score
  f1_total=f1_total+f1_score
  print(f"Total Precision: {p_total},recall_total:{recall_total},f1_total:{f1_total}")

BERTScore Precision: 0.7939835786819458, Recall: 0.7732957601547241, F1: 0.7835031151771545
Total Precision: 0.7939835786819458,recall_total:0.7732957601547241,f1_total:0.7835031151771545
BERTScore Precision: 0.7982280254364014, Recall: 0.790176272392273, F1: 0.7941817045211792
Total Precision: 1.5922116041183472,recall_total:1.563472032546997,f1_total:1.5776848793029785
BERTScore Precision: 0.7836920619010925, Recall: 0.7888133525848389, F1: 0.7862443327903748
Total Precision: 2.375903606414795,recall_total:2.352285385131836,f1_total:2.363929271697998
BERTScore Precision: 0.8164221048355103, Recall: 0.7724380493164062, F1: 0.7938212752342224
Total Precision: 3.1923255920410156,recall_total:3.124723434448242,f1_total:3.1577506065368652
BERTScore Precision: 0.7711414098739624, Recall: 0.7623295783996582, F1: 0.7667102217674255
Total Precision: 3.9634671211242676,recall_total:3.8870530128479004,f1_total:3.9244608879089355
BERTScore Precision: 0.7974412441253662, Recall: 0.799679040908813

In [ ]:
average_precision=p_total/len(answer)
average_recall=recall_total/len(answer)
average_f1=f1_total/len(answer)
print(f"Average Precision: {average_precision:4f},Average Recall:{average_recall:4f},Average F1 Score:{average_f1:4f}")

Average Precision: 0.805955,Average Recall:0.769707,Average F1 Score:0.787107
